In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats
from scipy.stats import norm
import scipy.integrate as integrate

import gym
from gym import spaces

import random
import itertools as it
from itertools import product
from joblib import Parallel, delayed
from toolz import memoize
from contracts import contract
from collections import namedtuple, defaultdict, deque, Counter

import warnings
warnings.filterwarnings("ignore", 
                        message="The objective has been evaluated at this point before.")

from agents import Agent
from oldmouselab import OldMouselabEnv
from policies import FixedPlanPolicy, LiederPolicy
from evaluation import *
from omdc_util import *
from distributions import cmax, smax, sample, expectation, Normal, PointMass, SampleDist, Normal, Categorical

In [2]:
def hd_dist(attributes):
    dist = [1,]*attributes
    dist[0] = np.random.randint(85,97)
    for i in range(1,attributes-1):
        dist[i] += np.random.randint(0,100-np.sum(dist))
    dist[-1] += 100-np.sum(dist)
    dist = np.around(np.array(dist)/100,decimals=2)
    np.random.shuffle(dist)
    return dist

def ld_dist(attributes):
    constrain = True
    while constrain:
        dist = [np.random.randint(10,50) for _ in range(attributes)]
        dist = np.around(np.array(dist)/sum(dist),decimals=2)
        constrain = np.min(dist) <= 0.10 or np.max(dist) >= 0.40
    np.random.shuffle(dist)
    return dist

In [3]:
gambles = 7
attributes = 4
high_stakes = Normal((9.99+0.01)/2, 0.3*(9.99-0.01))
low_stakes = Normal((0.25+0.01)/2, 0.3*(0.25-0.01))
reward = high_stakes
cost=.03

#set to 20 for sanity check
n_train = 20
n_test = 20

train_envs_hd = [OldMouselabEnv(gambles, hd_dist(attributes), reward, cost) for _ in range(n_train)]
train_envs_ld = [OldMouselabEnv(gambles, ld_dist(attributes), reward, cost) for _ in range(n_train)]
train_envs = train_envs_hd+train_envs_ld 

test_envs_hd =  [OldMouselabEnv(gambles, hd_dist(attributes), reward, cost) for _ in range(n_train)]
test_envs_ld = [OldMouselabEnv(gambles, ld_dist(attributes), reward, cost) for _ in range(n_train)]
test_envs = test_envs_hd+test_envs_ld 

term_action = train_envs[0].term_action

In [4]:
bo_pol_theta = np.load('data/om_bmps_pols/best/hs_hd_1cents.npy')
bo_pol = LiederPolicy(list(bo_pol_theta))

In [5]:
agent = Agent()
def run_env(policy, env):
    agent.register(env)
    agent.register(policy)
    tr = agent.run_episode()
#     print(tr)
    return {'util': tr['return'], 'actions': tr['actions'],
            'observations': len(tr['actions']) - 1, 'ground_truth': env.ground_truth}

def action_coordinate(env, action):
    return (action//env.outcomes,action%env.outcomes)

def p_grid(env, actions):
    grid = np.zeros((env.gambles+1,env.outcomes))
    grid[0,:] = env.dist
    for i in range(len(actions[:-1])):
        gamble, outcome = action_coordinate(env,actions[i]) 
        grid[gamble+1, outcome] = i+1
    return grid

# BMPS Run

In [6]:
train_envs[21].reset()
trace = run_env(bo_pol, train_envs[21])
trace

{'actions': [0, 3, 4, 7, 8, 12, 15, 16, 28],
 'ground_truth': array([  7.679,   9.24 ,  11.971,   3.385,   5.044,   4.185,   2.728,   2.096,   1.258,   9.381,   8.831,   2.364,   5.803,   5.457,   6.962,  10.345,   3.931,   2.917,   9.617,   7.885,   3.424,
          2.337,   7.598,   4.49 ,   3.41 ,   4.595,   1.553,  -0.706]),
 'observations': 8,
 'util': 6.9119645517827539}

In [7]:
train_envs[21].dist

array([ 0.35,  0.16,  0.14,  0.35])

In [8]:
train_envs[21].grid()

array([[7.6787260015816541, Norm(5.00, 2.99), Norm(5.00, 2.99), 3.3850064044943808],
       [5.0435050343933483, Norm(5.00, 2.99), Norm(5.00, 2.99), 2.0964847338403145],
       [1.2582014318099271, Norm(5.00, 2.99), Norm(5.00, 2.99), Norm(5.00, 2.99)],
       [5.8032041929612586, Norm(5.00, 2.99), Norm(5.00, 2.99), 10.345265954989468],
       [3.9312095602020438, Norm(5.00, 2.99), Norm(5.00, 2.99), Norm(5.00, 2.99)],
       [Norm(5.00, 2.99), Norm(5.00, 2.99), Norm(5.00, 2.99), Norm(5.00, 2.99)],
       [Norm(5.00, 2.99), Norm(5.00, 2.99), Norm(5.00, 2.99), Norm(5.00, 2.99)]], dtype=object)

In [9]:
p_grid(train_envs[21],trace['actions'])

array([[ 0.35,  0.16,  0.14,  0.35],
       [ 1.  ,  0.  ,  0.  ,  2.  ],
       [ 3.  ,  0.  ,  0.  ,  4.  ],
       [ 5.  ,  0.  ,  0.  ,  0.  ],
       [ 6.  ,  0.  ,  0.  ,  7.  ],
       [ 8.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ]])

# DC Run

In [10]:
train_envs[21].reset()
trace = run_dc(train_envs[21])
trace

{'actions': [0, 3, 23, 16, 15, 12, 13, 14, 28],
 'ground_truth': array([  7.679,   9.24 ,  11.971,   3.385,   5.044,   4.185,   2.728,   2.096,   1.258,   9.381,   8.831,   2.364,   5.803,   5.457,   6.962,  10.345,   3.931,   2.917,   9.617,   7.885,   3.424,
          2.337,   7.598,   4.49 ,   3.41 ,   4.595,   1.553,  -0.706]),
 'observations': 8,
 'options': [(0, 1),
  (0, 1),
  (5, 1),
  (4, 1),
  (3, 1),
  (3, 1),
  (3, 1),
  (3, 1),
  (-99, 1)],
 'util': 7.2597698719604011}

In [11]:
train_envs[21].dist

array([ 0.35,  0.16,  0.14,  0.35])

In [12]:
train_envs[21].grid()

array([[7.6787260015816541, Norm(5.00, 2.99), Norm(5.00, 2.99), 3.3850064044943808],
       [Norm(5.00, 2.99), Norm(5.00, 2.99), Norm(5.00, 2.99), Norm(5.00, 2.99)],
       [Norm(5.00, 2.99), Norm(5.00, 2.99), Norm(5.00, 2.99), Norm(5.00, 2.99)],
       [5.8032041929612586, 5.4572432343546655, 6.9617600191492883, 10.345265954989468],
       [3.9312095602020438, Norm(5.00, 2.99), Norm(5.00, 2.99), Norm(5.00, 2.99)],
       [Norm(5.00, 2.99), Norm(5.00, 2.99), Norm(5.00, 2.99), 4.4898446758034884],
       [Norm(5.00, 2.99), Norm(5.00, 2.99), Norm(5.00, 2.99), Norm(5.00, 2.99)]], dtype=object)

In [13]:
p_grid(train_envs[21],trace['actions'])

array([[ 0.35,  0.16,  0.14,  0.35],
       [ 1.  ,  0.  ,  0.  ,  2.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ],
       [ 6.  ,  7.  ,  8.  ,  5.  ],
       [ 4.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  3.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ]])

# Parsing

In [264]:
def make_env(gambles=7, cost=.01, ground_truth=False, dist=hd_dist(3), stakes = 'high'):
    reward = Normal((9.99+0.01)/2, 0.3*(9.99-0.01)) if stakes == 'high' else Normal((0.25+0.01)/2, 0.3*(0.25-0.01))
    return OldMouselabEnv(gambles, dist, reward, cost, ground_truth= ground_truth) 

In [265]:
env = train_envs[21]

In [266]:
env2 = make_env(ground_truth=env.ground_truth,dist=env.dist)

In [267]:
env.ground_truth

array([  7.679,   9.24 ,  11.971,   3.385,   5.044,   4.185,   2.728,   2.096,   1.258,   9.381,   8.831,   2.364,   5.803,   5.457,   6.962,  10.345,   3.931,   2.917,   9.617,   7.885,   3.424,
         2.337,   7.598,   4.49 ,   3.41 ,   4.595,   1.553,  -0.706])

In [268]:
env2.ground_truth

array([  7.679,   9.24 ,  11.971,   3.385,   5.044,   4.185,   2.728,   2.096,   1.258,   9.381,   8.831,   2.364,   5.803,   5.457,   6.962,  10.345,   3.931,   2.917,   9.617,   7.885,   3.424,
         2.337,   7.598,   4.49 ,   3.41 ,   4.595,   1.553,  -0.706])

In [269]:
trace = run_env(bo_pol, env)
trace

{'actions': [0, 3, 4, 7, 8, 12, 15, 16, 28],
 'ground_truth': array([  7.679,   9.24 ,  11.971,   3.385,   5.044,   4.185,   2.728,   2.096,   1.258,   9.381,   8.831,   2.364,   5.803,   5.457,   6.962,  10.345,   3.931,   2.917,   9.617,   7.885,   3.424,
          2.337,   7.598,   4.49 ,   3.41 ,   4.595,   1.553,  -0.706]),
 'observations': 8,
 'util': 6.9119645517827539}

In [20]:
trace = run_env(bo_pol, env2)
trace

{'actions': [0, 3, 4, 7, 8, 12, 15, 16],
 'ground_truth': array([  7.679,   9.24 ,  11.971,   3.385,   5.044,   4.185,   2.728,   2.096,   1.258,   9.381,   8.831,   2.364,   5.803,   5.457,   6.962,  10.345,   3.931,   2.917,   9.617,   7.885,   3.424,
          2.337,   7.598,   4.49 ,   3.41 ,   4.595,   1.553,  -0.706]),
 'observations': 7,
 'util': 7.0819645517827539}

In [452]:
def wrap_po(env,click_sequence,p_rand=0):
    memo = dict()
    def parse_options_clean(init_state,dist,cost,stakes,pre_acts,click_sequence):
        if click_sequence == []: 
            return True, [[]], [1]
        
        if (tuple(pre_acts),tuple(click_sequence)) in memo:       
            return memo[(tuple(pre_acts),tuple(click_sequence))]   
        
        envc = make_env(gambles=int(len(init_state)/len(dist)), ground_truth=init_state, cost=cost, dist=dist, stakes=stakes)
        envc.reset()
        for a in pre_acts:
            envc._step(a)

        option_seqs = []
        likelihoods = []
        done = False
        options, option_insts, option_utils, n_available_clicks,_,_,_ = get_all_options(envc)
        
        for i,j in product(range(1,min(len(dist),len(click_sequence))+1),range(len(options))):
            option = options[j]
            n_insts = len(option_insts[option])
            alpha = 1.0 if option == (-1,1) else 0.0
            beta = 1.0/np.sum(option_utils == np.max(option_utils)) if option_utils[j] == np.max(option_utils) else 0.0
            for inst in option_insts[option]:
                if np.array_equal(click_sequence[:i],inst): 
                    will_done, remaining, rem_likelihoods = (parse_options_clean(init_state,dist,cost,stakes,pre_acts+click_sequence[:i],click_sequence[i:]))
                    done = done or will_done  
                    if done:
                        for k in range(len(remaining)):
                            option_seqs.append([option]+remaining[k]) 
                            l_opt_seq = (1-p_rand)*beta + p_rand*alpha
                            likelihoods.append(l_opt_seq*rem_likelihoods[k]/n_insts)               
        memo[(tuple(pre_acts),tuple(click_sequence))] = done, option_seqs, likelihoods
        return done, option_seqs, likelihoods
    stakes = 'high' if env.reward.mu == 5.0 else 'low'
    if click_sequence[-1] != env.term_action:
        click_sequence = click_sequence+[env.term_action]
    return parse_options_clean(env.ground_truth,env.dist,env.cost,stakes,[],click_sequence)

In [453]:
env = OldMouselabEnv(4, ld_dist(3), reward, cost)

In [454]:
env.reset()
trace = run_dc(env)
click_seq = trace['actions']
click_seq

[4, 5, 1, 10, 11, 7, 8, 6, 3, 9, 12]

In [455]:
trace['options']

[(1, 1),
 (1, 1),
 (0, 1),
 (3, 1),
 (3, 1),
 (2, 1),
 (2, 1),
 (2, 1),
 (1, 1),
 (3, 1),
 (-99, 1)]

In [456]:
option_utils

array([-0.008,  0.001,  0.002, -0.008,  0.001,  0.002, -0.008,  0.001,  0.002,   -inf,  0.   ])

In [457]:
a,b,c = wrap_po(env,click_seq,p_rand=0)

In [458]:
np.max(c)

0.041666666666666664

In [459]:
list(zip(list(np.array(b)[np.array(c)!=0]),np.array(c)[np.array(c)!=0]))

[([(1, 1),
   (1, 1),
   (0, 1),
   (3, 1),
   (3, 1),
   (2, 1),
   (2, 1),
   (2, 1),
   (1, 1),
   (3, 1),
   (-99, 1)],
  0.041666666666666664)]

In [460]:
def dc_log_likelihood(env, trial, p_error):
    done, option_seqs, likelihoods = wrap_po(env,trial,p_rand=p_error)
    return np.log(np.sum(likelihoods))

In [461]:
def wrap_sum_ll(p, p_error):
    data = participants[p]
    return sum(dc_log_likelihood(env, trial, p_error)
                    for env,trial in data)

# Apply to Data

In [407]:
import json
df = pd.read_csv('../risky_choice/data/03242018/trials.csv', index_col=0)
for col in ['outcome_probs', 'ground_truth', 'clicks']:
    df[col] = df[col].apply(json.loads)

In [408]:
df

,clicks,decision,ground_truth,outcome_probs,reward_mu,reward_sigma,trial_index,workerid
0,[],1,"[0.16, 0.2, 0.16, 0.07, 0.1, 0.2, 0.12, 0.11, ...","[0.16, 0.19, 0.39, 0.26]",0.13,0.072,0,A2HSCKH5NKN5LP
1,[],6,"[0.08, 0.18, 0.14, 0.16, 0.03, 0.14, 0.01, 0.1...","[0.35, 0.27, 0.24, 0.14]",0.13,0.072,1,A2HSCKH5NKN5LP
2,[],4,"[0.11, 0.2, 0.17, 0.08, 0.18, 0.16, 0.12, 0.03...","[0.06, 0.05, 0.86, 0.03]",0.13,0.072,2,A2HSCKH5NKN5LP
3,[],4,"[0.06, 0.17, 0.19, 0.14, 0.07, 0.18, 0.15, 0.0...","[0.02, 0.9, 0.02, 0.06]",0.13,0.072,3,A2HSCKH5NKN5LP
4,[],5,"[0.09, 0.09, 0.11, 0.13, 0.2, 0.09, 0.24, 0.08...","[0.08, 0.9, 0.01, 0.01]",0.13,0.072,4,A2HSCKH5NKN5LP
5,[],2,"[0.03, 0.1, 0.02, 0.13, 0.1, 0.11, 0.16, 0.03,...","[0.01, 0.01, 0.07, 0.91]",0.13,0.072,5,A2HSCKH5NKN5LP
6,[],6,"[0.15, 0.08, 0.18, 0.01, 0.06, 0.2, 0.1, 0.2, ...","[0.3, 0.23, 0.25, 0.22]",0.13,0.072,6,A2HSCKH5NKN5LP
7,[],5,"[0.12, 0.22, 0.16, 0.13, 0.08, 0.05, 0.08, 0.1...","[0.32, 0.11, 0.33, 0.24]",0.13,0.072,7,A2HSCKH5NKN5LP
8,[],5,"[0.09, 0.12, 0.15, 0.16, 0.16, 0.02, 0.19, 0.0...","[0.28, 0.12, 0.37, 0.23]",0.13,0.072,8,A2HSCKH5NKN5LP
9,[15],4,"[0.09, 0.1, 0.09, 0.19, 0.16, 0.1, 0.05, 0.12,...","[0.04, 0.05, 0.06, 0.85]",0.13,0.072,9,A2HSCKH5NKN5LP


In [432]:
j = 0
participants = dict()
for i, row in df.iterrows():
    stakes = 'high' if env.reward.mu == 5.0 else 'low'
    n_gambles = int(len(row['ground_truth'])/len(row['outcome_probs']))
    env = make_env(gambles=n_gambles, ground_truth=row['ground_truth'], cost=0.01, dist=row['outcome_probs'], stakes=stakes)
    if row['workerid'] in participants.keys():
        participants[row['workerid']].append((env,row['clicks']))
    else:
        participants[row['workerid']] =[(env,row['clicks'])]

In [433]:
data = participants[p] 

In [462]:
data

[(<oldmouselab.OldMouselabEnv at 0x7c09da9e0198>, [5, 21, 25, 13, 15, 27]),
 (<oldmouselab.OldMouselabEnv at 0x7c09da9e00f0>, [0, 2, 12, 14, 24, 26]),
 (<oldmouselab.OldMouselabEnv at 0x7c09da9e05f8>, [3, 1]),
 (<oldmouselab.OldMouselabEnv at 0x7c09da9e07b8>, [1, 21, 23, 3]),
 (<oldmouselab.OldMouselabEnv at 0x7c09da9e08d0>, [26, 14, 18, 6]),
 (<oldmouselab.OldMouselabEnv at 0x7c09da9e0828>, [5, 13, 21, 25]),
 (<oldmouselab.OldMouselabEnv at 0x7c09da9e0048>, [5, 7, 9, 11, 25, 27]),
 (<oldmouselab.OldMouselabEnv at 0x7c09da9e06d8>, [5, 4]),
 (<oldmouselab.OldMouselabEnv at 0x7c09da9e07f0>, [6, 2, 18, 26]),
 (<oldmouselab.OldMouselabEnv at 0x7c09da9e04e0>, [6, 14, 18, 26]),
 (<oldmouselab.OldMouselabEnv at 0x7c09da9e02b0>, [3, 2, 14, 15, 27, 26]),
 (<oldmouselab.OldMouselabEnv at 0x7c09da9e0b70>, [1, 5, 9, 13, 17, 21, 25]),
 (<oldmouselab.OldMouselabEnv at 0x7c09da9e0080>,
  [5, 7, 15, 27, 25, 13, 4, 12, 16, 9, 24]),
 (<oldmouselab.OldMouselabEnv at 0x7c09da9e03c8>,
  [3, 7, 15, 12, 13, 

In [469]:
r_lls = []
parts = list(participants.keys())

In [470]:
for p in parts:
    print(p)
    r_lls.append(wrap_sum_ll(p, 1)) 

A2HSCKH5NKN5LP
A8RDXT4ZILHQT


KeyboardInterrupt: 

In [467]:
import copy

In [484]:
sm = []
probses = []
for p in parts:
    data = participants[p]
    probs = []
    for episode in data:
        env, trial = copy.deepcopy(episode)
        trial = trial+[env.term_action]
        prob = 1
        for a in trial:
            prob*=1/len(list(env.actions(env._state)))
            env.step(a)
        probs.append(prob)
    probses.append(probs)
    sm.append(np.log(np.prod(probs,dtype=np.float128)))

In [485]:
sm

[-186.55834681360302311,
 -415.12751293373600731,
 -70.678121109904684723,
 -269.90022346988832957,
 -332.40046656027136032,
 -544.22860194363583153,
 -367.93190724044591827,
 -604.88842103788115451,
 -699.18599455266860304,
 -368.96539752421140998,
 -477.4026441600996836,
 -264.48320406059489088,
 -309.06162849099231921,
 -362.6437923369705536,
 -437.20247861513427032,
 -635.32149497810598932,
 -518.48020379822771114,
 -241.26216802811697945,
 -299.60304601143091582,
 -811.7387036083828849,
 -423.62852893787495362,
 -369.38272348910061471,
 -512.82979066147810449,
 -878.78139886530327196,
 -775.38099358642899139,
 -353.0087121339904298,
 -571.39323974953368851,
 -348.52075937464472974,
 -651.34221470543643345,
 -415.88107843705621675,
 -357.1071282779744463,
 -242.18651846163371365,
 -536.46259057272865284,
 -695.00122544843192379,
 -67.345916599729480821,
 -199.06219041377048633,
 -67.345916599729480821,
 -474.68551317219129951,
 -556.77453242340562278,
 -603.44330387224859058,
 -590

In [490]:
np.sum(sm)

-44586.332420088116766

In [491]:
[dc_log_likelihood(env, trial, p_error) for env,trial in participants[parts[93]]]

KeyboardInterrupt: 

In [475]:
def mle(data):
# You can adjust the temperature bounds if you think the MLE
# is not in the bounds below. Don't change the p_error bounds.
    bounds = [
        (0, 0.25)  # p_error
    ]
    def loss(x):
        temp, p_error = x
        return -sum(dc_log_likelihood(env, trial, p_error)
                    for env, trial in data)
    res = scipy.optimize.minimize(loss, (.01,),bounds = bounds,
                                 options={'maxiter':100,'disp':True})
    p_error = res.x
    return {'p_error': res.x[1], 'logp': -res.fun}

In [ ]:
def mle_wrap(p):
    print(p)
    d = mle(participants[p])
    d['participant'] = p
    return d

In [ ]:
mles = Parallel(n_jobs=60)(delayed(mle_wrap)(p)
                           for p in participants.keys())